# Lab 03: LSTM with Pytorch - exercise

In [ ]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    path_to_file = '/content/gdrive/My Drive/CS5242_2025_codes/labs_lecture06/lab03_lstm_pytorch'
    print(path_to_file)
    # change current path to the folder containing "file_name"
    os.chdir(path_to_file)
    !pwd

In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
import time
import utils

### With or without GPU?

It is recommended to run this code on GPU:<br> 
* Time for 1 epoch on CPU : 274 sec ( 4.56 min)<br> 
* Time for 1 epoch on GPU : 10.1 sec w/ GeForce GTX 1080 Ti <br>

In [3]:
device= torch.device("cuda")
device= torch.device("cpu")
print(device)

if torch.cuda.is_available():
    print('cuda available with GPU:',torch.cuda.get_device_name(0))

cpu


### Download Penn Tree Bank

The tensor train_data consists of 20 columns of 46,479 words.<br>
The tensor test_data consists of 20 columns of 4,121 words.

In [4]:
from utils import check_ptb_dataset_exists
data_path=check_ptb_dataset_exists()

train_data  =  torch.load(data_path+'ptb/train_data.pt')
test_data   =  torch.load(data_path+'ptb/test_data.pt')

print(  train_data.size()  )
print(  test_data.size()   )

torch.Size([46479, 20])
torch.Size([4121, 20])


### Some constants associated with the data set

In [5]:
bs = 20

vocab_size = 10000


### Make a LTSM class

Step 1: Embed the discrete words into a continuous space using PyTorch’s nn.Embedding().

Step 2: Process the embedded words with a LSTM layer using PyTorch’s nn.LSTM().

Step 3: Project the processed tokens back into the vocabulary space by outputting a score vector over the vocabulary set with PyTorch’s nn.Linear().



In [6]:
class three_layer_recurrent_net(nn.Module):

    def __init__(self, hidden_size):
        super(three_layer_recurrent_net, self).__init__()
        
        self.layer1 = nn.Embedding(vocab_size, hidden_size)# COMPLETE HERE 
        self.layer2 = nn.LSTM(hidden_size, hidden_size)# COMPLETE HERE 
        self.layer3 = nn.Linear(hidden_size, vocab_size)# COMPLETE HERE 

        
    def forward(self, word_seq, h_init, c_init ):
        
        g_seq                      =  self.layer1(word_seq) # COMPLETE HERE 
        h_seq , (h_final,c_final)  =  self.layer2(g_seq, (h_init, c_init)) # COMPLETE HERE 
        score_seq                  =  self.layer3(h_seq) # COMPLETE HERE 
        
        return score_seq,  h_final , c_final


### Build the net. Choose the hidden size to be 300. How many parameters in total?

In [7]:
hidden_size=300

net = three_layer_recurrent_net( hidden_size )

print(net)

utils.display_num_param(net)

three_layer_recurrent_net(
  (layer1): Embedding(10000, 300)
  (layer2): LSTM(300, 300)
  (layer3): Linear(in_features=300, out_features=10000, bias=True)
)
There are 6732400 (6.73 million) parameters in this neural network


### Send the weights of the networks to the GPU

In [8]:
net = net.to(device)

### Set up manually the weights of the embedding module and Linear module

In [9]:
net.layer1.weight.data.uniform_(-0.1, 0.1)

net.layer3.weight.data.uniform_(-0.1, 0.1)

print('')

### Choose the criterion, as well as the following important hyperparameters: 
* initial learning rate = 5
* sequence length = 35

In [10]:
criterion = nn.CrossEntropyLoss()

my_lr = 5

seq_length = 35

### Function to evaluate the network on the test set

In [11]:
def eval_on_test_set():

    running_loss=0
    num_batches=0    
       
    h = torch.zeros(1, bs, hidden_size)
    c = torch.zeros(1, bs, hidden_size)
   
    h=h.to(device)
    c=c.to(device)
       
    for count in range( 0 , 4120-seq_length ,  seq_length) :
               
        minibatch_data =  test_data[ count   : count+seq_length   ]
        minibatch_label = test_data[ count+1 : count+seq_length+1 ]
        
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
                                  
        scores, h, c  = net( minibatch_data, h , c)
        
        minibatch_label =   minibatch_label.view(  bs*seq_length ) 
        scores          =            scores.view(  bs*seq_length , vocab_size)
        
        loss = criterion(  scores ,  minibatch_label )    
        
        h=h.detach()
        c=c.detach()
            
        running_loss += loss.item()
        num_batches += 1        
    
    total_loss = running_loss/num_batches 
    print('test: exp(loss) = ', math.exp(total_loss)  )
        

### Do 8 passes through the training set.

In [12]:
start=time.time()

for epoch in range(8):
    
    # divide the learning rate by 3 except after the first epoch
    if epoch >= 2:
        my_lr = my_lr / 3
        # COMPLETE HERE 
    
    # create a new optimizer at the beginning of each epoch: give the current learning rate.   
    optimizer=torch.optim.SGD( net.parameters() , lr=my_lr )
        
    # set the running quatities to zero at the beginning of the epoch
    running_loss=0
    num_batches=0    
       
    # set the initial h and c to be the zero vector
    h = torch.zeros(1, bs, hidden_size)
    c = torch.zeros(1, bs, hidden_size)

    # send them to the gpu    
    h=h.to(device)
    c=c.to(device)
    
    for count in range( 0 , 46478-seq_length ,  seq_length):
        
        # Set the gradients to zeros
        optimizer.zero_grad()
        
        # create a minibatch
        minibatch_data =  train_data[ count   : count+seq_length   ]
        minibatch_label = train_data[ count+1: count+seq_length + 1] # COMPLETE HERE 
        
        # send them to the gpu
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
        
        # Detach to prevent from backpropagating all the way to the beginning
        # Then tell Pytorch to start tracking all operations that will be done on h and c
        h= h.detach()# COMPLETE HERE 
        c= c.detach()# COMPLETE HERE 
        h= h.requires_grad_()# COMPLETE HERE 
        c= c.requires_grad_()# COMPLETE HERE 
                       
        # forward the minibatch through the net        
        scores, h, c  = net( minibatch_data, h , c)
        
        # reshape the scores and labels to huge batch of size bs*seq_length
        scores          =  scores.view(bs*seq_length, vocab_size)# COMPLETE HERE 
        minibatch_label =  minibatch_label.view(bs*seq_length)# COMPLETE HERE 
        
        # Compute the average of the losses of the data points in this huge batch
        loss = criterion(  scores ,  minibatch_label )
        
        # backward pass to compute dL/dR, dL/dV and dL/dW
        loss.backward()

        # do one step of stochastic gradient descent: R=R-lr(dL/dR), V=V-lr(dL/dV), ...
        utils.normalize_gradient(net)
        optimizer.step()
        
        # update the running loss  
        running_loss += loss.item()
        num_batches += 1
        
        
        
    # compute stats for the full training set
    total_loss = running_loss/num_batches
    elapsed = time.time()-start
    
    print('')
    print('epoch=',epoch, '\t time=', elapsed,'\t lr=', my_lr, '\t exp(loss)=',  math.exp(total_loss))
    eval_on_test_set() 



epoch= 0 	 time= 269.7891972064972 	 lr= 5 	 exp(loss)= 278.3187498502871
test: exp(loss) =  174.7514116699017

epoch= 1 	 time= 551.3592555522919 	 lr= 5 	 exp(loss)= 126.36804189480901
test: exp(loss) =  133.72842667211165

epoch= 2 	 time= 821.4820165634155 	 lr= 1.6666666666666667 	 exp(loss)= 80.93701796928406
test: exp(loss) =  114.19627872735583

epoch= 3 	 time= 1103.4953067302704 	 lr= 0.5555555555555556 	 exp(loss)= 66.84685233678589
test: exp(loss) =  110.23330082203049

epoch= 4 	 time= 1361.1420533657074 	 lr= 0.1851851851851852 	 exp(loss)= 62.02632212033655
test: exp(loss) =  108.8377668344424

epoch= 5 	 time= 1623.113030910492 	 lr= 0.0617283950617284 	 exp(loss)= 60.292681086419314
test: exp(loss) =  108.08784381604626

epoch= 6 	 time= 1893.3254716396332 	 lr= 0.0205761316872428 	 exp(loss)= 59.66023543857577
test: exp(loss) =  107.55508685862928

epoch= 7 	 time= 2157.3052093982697 	 lr= 0.006858710562414266 	 exp(loss)= 59.43120207875743
test: exp(loss) =  107.243

### Choose one sentence (taken from the test set)

In [13]:
# SENTENCES FROM TEST SET

sentence1 = "some analysts expect oil prices to remain relatively"

sentence2 = "over the next days and weeks they say investors should look for stocks to"

sentence2 = "some analysts expect oil prices to remain relatively"

sentence3 = "prices averaging roughly $ N a barrel higher in the third"

sentence4 = "i think my line has been very consistent mrs. hills said at a news"

sentence5 = "this appears particularly true at gm which had strong sales in"

# OR MAKE YOUR OWN SENTENCE. 
# NO CAPITAL LETTER ALLOWED. EACH WORD MUST BE IN THE ALLOWED VOCABULARY OF 10,000 WORDS

sentence6= "he was very"


# CHOOSE THE SENTENCE
mysentence = sentence3

### Convert the sentence into a vector, then send to GPU

In [14]:
minibatch_data=utils.sentence2vector(mysentence)
      
minibatch_data=minibatch_data.to(device)

print(minibatch_data)

tensor([[1786],
        [8705],
        [3246],
        [ 416],
        [  27],
        [  35],
        [2664],
        [ 209],
        [ 108],
        [  32],
        [3017]])


### Set the initial hidden state to zero, then run the LSTM.

In [15]:
h = torch.zeros(1, 1, hidden_size)
c = torch.zeros(1, 1, hidden_size)
h=h.to(device)
c=c.to(device)

scores , h, c = net(minibatch_data , h, c)

### Display the network prediction for the next word

In [16]:
print(mysentence, '... \n')

utils.show_next_word(scores)

prices averaging roughly $ N a barrel higher in the third ... 

93.9%	 quarter
1.4%	 period
0.8%	 consecutive
0.4%	 week
0.4%	 year
0.2%	 third
0.2%	 half
0.2%	 month
0.1%	 game
0.1%	 quarters
0.1%	 day
0.1%	 world
0.1%	 fourth
0.1%	 floor
0.1%	 time
0.1%	 trading
0.0%	 house
0.0%	 market
0.0%	 and
0.0%	 business
0.0%	 area
0.0%	 part
0.0%	 of
0.0%	 line
0.0%	 <eos>
0.0%	 hour
0.0%	 largest
0.0%	 section
0.0%	 season
0.0%	 nine
